In [1]:
# Note: This is a hack to allow importing from the parent directory
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [2]:
from data import CIFAR10GaussianSplatsDataset

dataset = CIFAR10GaussianSplatsDataset(
    root="../data/CIFAR10GS",
    train=True,
    init_type="grid",
)
dataset[0]

(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32>,
 6,
 ParameterDict(
     (Ks): Parameter containing: [torch.FloatTensor of size 3x3]
     (means): Parameter containing: [torch.FloatTensor of size 1024x3]
     (opacities): Parameter containing: [torch.FloatTensor of size 1024]
     (quats): Parameter containing: [torch.FloatTensor of size 1024x4]
     (scales): Parameter containing: [torch.FloatTensor of size 1024x3]
     (viewmats): Parameter containing: [torch.FloatTensor of size 4x4]
     (colors): Parameter containing: [torch.FloatTensor of size 1024x4x3]
 ))

In [3]:
from utils import transform_autoencoder_input, transform_autoencoder_output

print(transform_autoencoder_input(dataset[0][-1]))
print(transform_autoencoder_output(transform_autoencoder_input(dataset[0][-1])))

tensor([-1., -1.,  0.,  ...,  0.,  0.,  0.], grad_fn=<CatBackward0>)
ParameterDict(
    (Ks): Parameter containing: [torch.FloatTensor of size 3x3]
    (colors): Parameter containing: [torch.FloatTensor of size 1024x4x3]
    (means): Parameter containing: [torch.FloatTensor of size 1024x3]
    (opacities): Parameter containing: [torch.FloatTensor of size 1024]
    (quats): Parameter containing: [torch.FloatTensor of size 1024x4]
    (scales): Parameter containing: [torch.FloatTensor of size 1024x3]
    (viewmats): Parameter containing: [torch.FloatTensor of size 4x4]
)


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import AbstractAutoencoder


class ShallowAutoencoder(AbstractAutoencoder):
    def __init__(self, input_dim, latent_dim, use_bias=True):
        super(ShallowAutoencoder, self).__init__()
        self.type = "ShallowAutoencoder"
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, latent_dim, bias=use_bias),
            nn.ReLU(inplace=True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim, bias=use_bias),
            nn.Sigmoid(),
        )